In [1]:
import dataextraction as de
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re,string,unicodedata
from string import punctuation
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rohang62/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = de.extract_data('data/train.jsonl')
train = de.parse_json(data, True)

data = de.extract_data('data/test.jsonl')
test = de.parse_json(data, False)

In [3]:
print(len(train.index))
print(len(test.index))

5000
1800


In [4]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [5]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i != "@USER":
            final_text.append(i.strip().lower())
    return " ".join(final_text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

train['response']=train['response'].apply(denoise_text)
test['response']=test['response'].apply(denoise_text)

In [6]:
train.head()

,label,response
0,SARCASM,get .. obviously care would've moved right alo...
0,SARCASM,trying protest talking labels label wtf make em
0,SARCASM,makes insane money movies einstein #learnhowth...
0,SARCASM,meanwhile trump even release sat scores wharto...
0,SARCASM,pretty sure anti-lincoln crowd claimed democra...


In [108]:
with open('train_ft.txt', 'w') as f:
    for each_text, each_label in zip(train['response'], train['label']):
        f.writelines(f'__label__{each_label} {each_text}\n')

In [153]:
import fasttext
model = fasttext.train_supervised('train_ft.txt', epoch=300, lr=1, wordNgrams=5, verbose=2, minCount=1)

In [154]:
def print_results(sample_size, precision, recall):
    precision   = round(precision, 2)
    recall      = round(recall, 2)
    print(f'{precision}')
    print(f'{recall}')

print_results(*model.test('train_ft.txt'))

1.0
1.0


In [155]:
def predict_is_sarcastic(text):
    return model.predict(text, k=2)

In [156]:
!rm answer.txt

In [157]:
with open('answer.txt', 'w') as f:
    for each_id, each_text in zip(test['id'], test['response']):
        labels, probs = predict_is_sarcastic(f'{each_text}')
        l = None
        if (probs[0] > probs[1] and labels[0] == "__label__SARCASM"):
            l = "SARCASM"
        elif (probs[0] > probs[1] and labels[0] != "__label__SARCASM"): 
            l = "NOT_SARCASM"
        elif (probs[0] < probs[1] and labels[0] == "__label__SARCASM"): 
            l = "NOT_SARCASM"
        else:
            l = "SARCASM"
        f.writelines(f'{each_id},{l}\n')

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [8]:
train = train.replace(1, "SARCASM")
train = train.replace(0, "NOT_SARCASM")

In [9]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [10]:
tokenized = train['response'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [11]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [12]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5000, 73)

In [13]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [14]:
train_features = last_hidden_states[0][:,0,:].numpy()
train_labels = train['label']

In [15]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [16]:
tokenized = test['response'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [17]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [18]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1800, 49)

In [19]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [20]:
test_features = last_hidden_states[0][:,0,:].numpy()
test_pred = lr_clf.predict(test_features)

In [21]:
with open('answer.txt', 'w') as f:
    for tid, pred in zip(test['id'], test_pred):
        l = None
        if pred == 1:
            l = "SARCASM"
        else:
            l = "NOT_SARCASM"
        f.writelines(f'{tid},{l}\n')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()